In [1]:
from pathlib import Path
import functools
import random
import json
import os

from spacy.tokens import DocBin
import spacy

In [2]:
training_path = Path('training/corpus')
source_path = training_path / 'source'
fixed_path = training_path / 'fixed'

for path in source_path.glob('*.txt'):
    txt = path.read_text(encoding='utf-8')
    txt = '\n'.join([t.strip() for t in txt.split('\n')])
    fpath = fixed_path / path.name
    fpath.write_text(txt, encoding='utf-8')
    os.system(f'python -m spacy convert {fpath} {training_path} --converter ner')

In [3]:
nlp = spacy.load('en_core_web_lg')

In [4]:
for path in training_path.glob('*.spacy'):
    doc_bin = DocBin().from_disk(path)
    docs = list(doc_bin.get_docs(nlp.vocab))
    print(path, ' : ', len(docs))

training\corpus\Final_SCIREX_dev.spacy  :  17871
training\corpus\Final_SCIREX_test.spacy  :  19429
training\corpus\Final_SCIREX_train.spacy  :  83132
training\corpus\tdm_dev.spacy  :  17871
training\corpus\tdm_test.spacy  :  19429
training\corpus\tdm_train.spacy  :  83132


In [5]:
to_keep = ['Material', 'Metric', 'Task']

labels = []
for path in training_path.glob('*.spacy'):
 
    input_bin = DocBin().from_disk(path)
    output_bin = DocBin(attrs=["ENT_IOB", "ENT_TYPE"])

    for doc in input_bin.get_docs(nlp.vocab):
        ents = [ent for ent in doc.ents if ent.label_ in to_keep]
        doc.ents = tuple(ents)
        output_bin.add(doc)

    output_path = training_path / f"tdm_{path.name.split('_')[-1]}"
    output_bin.to_disk(path)